# Desafío Alura Store: Análisis de ventas y rendimiento de las tiendas

Este cuaderno guía el análisis para ayudar al Sr. Juan (João) a decidir **qué tienda debería vender** para iniciar un nuevo emprendimiento. Se evaluarán cinco aspectos:

1. **Facturación total** por tienda.
2. **Categorías más populares** (más vendidas) por tienda.
3. **Calificación promedio** por tienda (a partir de reseñas).
4. **Productos más y menos vendidos** por tienda.
5. **Costo promedio de envío** por tienda.

**Visualizaciones:** se incluyen **al menos 3 tipos de gráficos diferentes** usando *matplotlib*: **barras**, **circular (pie)** y **dispersión (scatter)**.

> **Sugerencia:** Mantén intacta la sección de **Importación de datos** si ya cuentas con el *código base* provisto en Trello/GitHub. Si no lo tienes, usa la sección de **Configuración de datos** para apuntar a tu carpeta o archivos CSV.



## Índice
- [0) Preparación](#0)
- [1) Importación de datos (no modificar)](#1)
- [2) Configuración de datos (si no usas el código base)](#2)
- [3) Limpieza y estandarización](#3)
- [4) Análisis 1: Facturación total por tienda](#4)
- [5) Análisis 2: Categorías más populares por tienda](#5)
- [6) Análisis 3: Calificación promedio por tienda](#6)
- [7) Análisis 4: Productos más y menos vendidos por tienda](#7)
- [8) Análisis 5: Costo promedio de envío por tienda](#8)
- [9) Reporte final y recomendación](#9)
- [10) Cumplimiento de requisitos](#10)



<a id="0"></a>

## 0) Preparación
Instalación y carga de librerías mínimas. *Usamos matplotlib (sin estilos ni colores específicos) como pide el proyecto.*


In [ ]:

import os
import glob
import math
from typing import Dict, List

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')



<a id="1"></a>

## 1) Importación de datos (no modificar)

**Si estás usando el *código base* del desafío:** esta celda ya debería leer los datos correctos.
Si **NO** tienes el *código base*, **no ejecutes esta celda** y utiliza la sección de **Configuración de datos** (sección 2).


In [ ]:

# === IMPORTACIÓN DE DATOS DESDE CÓDIGO BASE (DEJAR TAL CUAL) ===
# from codigo_base import cargar_datos  # hipotético módulo del repositorio base
# df = cargar_datos()  # Debe devolver un DataFrame consolidado con todas las tiendas

# En caso de tener df ya cargado por el código base, asegúrate de que contenga al menos:
# ['store', 'order_id', 'product_id', 'product_name', 'category', 'quantity', 'price', 'review_score', 'shipping_cost']
# raise SystemExit('Si ya tienes df desde el código base, comenta este SystemExit.')



<a id="2"></a>

## 2) Configuración de datos (si no usas el código base)

Esta ruta permite **apuntar a una carpeta** con los CSV de las 4 tiendas **o** a un único CSV consolidado.
El cuaderno **intentará estandarizar** nombres de columnas comunes en español/inglés/portugués.


In [ ]:

# === OPCIÓN A: Carpeta con múltiples CSV, uno por tienda ===
DATA_DIR = 'data'  # cámbialo si tu carpeta tiene otro nombre
FILE_PATTERN = '*.csv'  # patrón para buscar archivos dentro de DATA_DIR

# === OPCIÓN B: Un solo CSV consolidado ===
# SINGLE_CSV = 'data/consolidado.csv'  # descomenta y usa si prefieres uno solo
SINGLE_CSV = None

# === MAPEO DE COLUMNAS: agrega aquí variantes que existan en tus archivos ===
COLUMN_ALIASES = {
    'store': ['store', 'tienda', 'seller_store', 'loja'],
    'order_id': ['order_id', 'id_pedido', 'pedido_id'],
    'product_id': ['product_id', 'id_producto', 'produto_id'],
    'product_name': ['product_name', 'nombre_producto', 'produto_nome', 'product'],
    'category': ['category', 'categoria', 'product_category', 'categoria_producto'],
    'quantity': ['quantity', 'cantidad', 'qty', 'qtd'],
    'price': ['price', 'precio', 'preco', 'unit_price'],
    'review_score': ['review_score', 'calificacion', 'nota', 'rating', 'review', 'score'],
    'shipping_cost': ['shipping_cost', 'costo_envio', 'frete', 'shipping_price', 'envio'],
    'customer_id': ['customer_id', 'id_cliente', 'cliente_id']
}

def _standardize_columns(df: pd.DataFrame, aliases: Dict[str, List[str]]) -> pd.DataFrame:
    cols = {c.lower().strip(): c for c in df.columns}
    result = {}
    for canonical, options in aliases.items():
        found = None
        for opt in options:
            if opt in cols:
                found = cols[opt]
                break
        if found is not None:
            result[found] = canonical
    return df.rename(columns=result)

def _read_many_csvs(data_dir: str, pattern: str) -> pd.DataFrame:
    paths = sorted(glob.glob(os.path.join(data_dir, pattern)))
    if not paths:
        raise FileNotFoundError(f'No se encontraron CSV con el patrón {pattern} en {data_dir}')
    frames = []
    for p in paths:
        tmp = pd.read_csv(p, low_memory=False)
        tmp = _standardize_columns(tmp, COLUMN_ALIASES)
        if 'store' not in tmp.columns:
            inferred = os.path.splitext(os.path.basename(p))[0]
            tmp['store'] = inferred
        frames.append(tmp)
    df = pd.concat(frames, ignore_index=True)
    return df

def _read_single_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, low_memory=False)
    df = _standardize_columns(df, COLUMN_ALIASES)
    if 'store' not in df.columns:
        raise ValueError('El CSV consolidado debe incluir una columna "store" que identifique la tienda.')
    return df

if 'df' not in globals():
    if SINGLE_CSV:
        df = _read_single_csv(SINGLE_CSV)
    else:
        df = _read_many_csvs(DATA_DIR, FILE_PATTERN)

# Chequeo mínimo de columnas requeridas
required = ['store', 'order_id', 'product_id', 'category', 'quantity', 'price', 'review_score', 'shipping_cost']
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f'Faltan columnas requeridas: {missing}\nColumnas disponibles: {list(df.columns)}')

if 'product_name' not in df.columns:
    df['product_name'] = df['product_id'].astype(str)

for c in ['quantity', 'price', 'review_score', 'shipping_cost']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df['revenue'] = df['quantity'] * df['price']
df['units'] = df['quantity']
df.sample(min(len(df), 5), random_state=42)



<a id="3"></a>

## 3) Limpieza y estandarización

- Eliminamos filas con información insuficiente para los indicadores.
- Reemplazamos valores negativos por NaN donde no tenga sentido.


In [ ]:

df_clean = df.dropna(subset=['store', 'order_id', 'product_id', 'category', 'quantity', 'price']).copy()

for c in ['quantity', 'price', 'shipping_cost', 'review_score']:
    if c in df_clean.columns:
        df_clean.loc[df_clean[c] < 0, c] = np.nan

stores = sorted(df_clean['store'].dropna().unique().tolist())
print(f'Tiendas detectadas ({len(stores)}):', stores)
print(f'Registros válidos: {len(df_clean):,}')



<a id="4"></a>

## 4) Análisis 1: Facturación total por tienda

Incluye **gráfico de barras** y **gráfico circular (pie)** para participación de facturación.


In [ ]:

facturacion = (df_clean
               .groupby('store', as_index=False)
               .agg(revenue_total=('revenue', 'sum'),
                    unidades_total=('units', 'sum'))
               .sort_values('revenue_total', ascending=False))

display(facturacion)

# Barras
plt.figure()
plt.bar(facturacion['store'], facturacion['revenue_total'])
plt.title('Facturación total por tienda (Barras)')
plt.xlabel('Tienda')
plt.ylabel('Facturación total')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Circular (pie) - participación de facturación
plt.figure()
plt.pie(facturacion['revenue_total'], labels=facturacion['store'], autopct='%1.1f%%')
plt.title('Participación de facturación por tienda (Pie)')
plt.tight_layout()
plt.show()



<a id="5"></a>

## 5) Análisis 2: Categorías más populares por tienda

Gráficos de **barras** por tienda con el Top-N de categorías.


In [ ]:

top_n = 5
cats = (df_clean
        .groupby(['store', 'category'], as_index=False)
        .agg(unidades=('units', 'sum'))
        .sort_values(['store', 'unidades'], ascending=[True, False]))

display(cats.head(10))

for s in stores:
    sub = cats[cats['store'] == s].head(top_n)
    plt.figure()
    plt.bar(sub['category'].astype(str), sub['unidades'])
    plt.title(f'Top {top_n} categorías por unidades - {s} (Barras)')
    plt.xlabel('Categoría')
    plt.ylabel('Unidades vendidas')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()



<a id="6"></a>

## 6) Análisis 3: Calificación promedio por tienda

Incluye **barras** para promedio y **histograma** opcional de distribución de reseñas.


In [ ]:

if 'review_score' in df_clean.columns:
    ratings = (df_clean
               .dropna(subset=['review_score'])
               .groupby('store', as_index=False)
               .agg(calificacion_promedio=('review_score', 'mean'),
                    cantidad_resenas=('review_score', 'count'))
               .sort_values('calificacion_promedio', ascending=False))
    display(ratings)

    plt.figure()
    plt.bar(ratings['store'], ratings['calificacion_promedio'])
    plt.title('Calificación promedio por tienda (Barras)')
    plt.xlabel('Tienda')
    plt.ylabel('Calificación promedio')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # Histograma global de reseñas (tercer tipo de gráfico alternativo)
    plt.figure()
    df_clean['review_score'].dropna().plot(kind='hist', bins=10)
    plt.title('Distribución de puntuaciones de reseñas (Histograma)')
    plt.xlabel('Puntuación')
    plt.ylabel('Frecuencia')
    plt.tight_layout()
    plt.show()
else:
    print('No existe la columna review_score en los datos.')



<a id="7"></a>

## 7) Análisis 4: Productos más y menos vendidos por tienda

Identificamos el **top 5** y **bottom 5** por unidades de cada tienda (barras).


In [ ]:

prod = (df_clean
        .groupby(['store', 'product_id', 'product_name'], as_index=False)
        .agg(unidades=('units', 'sum'),
             revenue=('revenue', 'sum')))

def extremos_por_tienda(df_in, k=5):
    resultados = []
    for s in stores:
        sub = df_in[df_in['store'] == s].sort_values('unidades', ascending=False)
        top = sub.head(k).assign(tipo='TOP')
        bottom = sub.sort_values('unidades', ascending=True).head(k).assign(tipo='BOTTOM')
        resultados.append(pd.concat([top, bottom], ignore_index=True))
    return pd.concat(resultados, ignore_index=True)

extremos = extremos_por_tienda(prod, k=5)
display(extremos)

for s in stores:
    sub = extremos[(extremos['store'] == s) & (extremos['tipo'] == 'TOP')]
    plt.figure()
    plt.bar(sub['product_name'].astype(str), sub['unidades'])
    plt.title(f'Productos TOP 5 por unidades - {s} (Barras)')
    plt.xlabel('Producto')
    plt.ylabel('Unidades')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()



<a id="8"></a>

## 8) Análisis 5: Costo promedio de envío por tienda

Incluye **barras** por tienda y un **gráfico de dispersión (scatter)** relacionando costo de envío y valor del pedido (muestra).


In [ ]:

envio = (df_clean
         .dropna(subset=['shipping_cost'])
         .groupby('store', as_index=False)
         .agg(costo_envio_promedio=('shipping_cost', 'mean'),
              costo_envio_mediano=('shipping_cost', 'median'),
              costo_envio_total=('shipping_cost', 'sum'))
         .sort_values('costo_envio_promedio', ascending=True))
display(envio)

plt.figure()
plt.bar(envio['store'], envio['costo_envio_promedio'])
plt.title('Costo promedio de envío por tienda (Barras)')
plt.xlabel('Tienda')
plt.ylabel('Costo promedio de envío')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Scatter: costo de envío vs. valor del pedido (muestra aleatoria para performance)
sample = df_clean.dropna(subset=['shipping_cost', 'revenue']).sample(
    min(len(df_clean), 5000), random_state=7
)
plt.figure()
plt.scatter(sample['revenue'], sample['shipping_cost'], alpha=0.5)
plt.title('Relación costo de envío vs valor del pedido (Scatter)')
plt.xlabel('Valor del pedido (revenue)')
plt.ylabel('Costo de envío')
plt.tight_layout()
plt.show()



<a id="9"></a>

## 9) Reporte final y recomendación

Construimos un **índice compuesto** para identificar la **tienda con peor desempeño** considerando:
- **Facturación** (a mayor, mejor).
- **Calificación promedio** (a mayor, mejor).
- **Costo de envío** (a menor, mejor).
- **Unidades vendidas** (a mayor, mejor).

> Puedes ajustar los **pesos** según el criterio del jefe/negocio.


In [ ]:

# Preparar métricas por tienda
base = facturacion[['store', 'revenue_total', 'unidades_total']].copy()

if 'ratings' in locals():
    base = base.merge(ratings[['store', 'calificacion_promedio']], on='store', how='left')
else:
    base['calificacion_promedio'] = np.nan

base = base.merge(envio[['store', 'costo_envio_promedio']], on='store', how='left')

def minmax(series):
    s = series.astype(float)
    if s.nunique(dropna=True) <= 1:
        return pd.Series([0.5 if not math.isnan(x) else np.nan for x in s], index=s.index)
    min_v, max_v = s.min(skipna=True), s.max(skipna=True)
    return (s - min_v) / (max_v - min_v)

norm_rev = minmax(base['revenue_total'])
norm_units = minmax(base['unidades_total'])
norm_rate = minmax(base['calificacion_promedio'])
norm_ship = minmax(-base['costo_envio_promedio'])

weights = {'revenue': 0.4, 'units': 0.2, 'rating': 0.25, 'shipping': 0.15}

base['score'] = (weights['revenue'] * norm_rev.fillna(norm_rev.mean()) +
                 weights['units']   * norm_units.fillna(norm_units.mean()) +
                 weights['rating']  * norm_rate.fillna(norm_rate.mean()) +
                 weights['shipping']* norm_ship.fillna(norm_ship.mean()))

resultado = base.sort_values('score', ascending=False).reset_index(drop=True)
resultado['ranking'] = np.arange(1, len(resultado) + 1)
display(resultado[['ranking','store','score','revenue_total','unidades_total','calificacion_promedio','costo_envio_promedio']])

peor_tienda = resultado.sort_values('score', ascending=True).iloc[0]['store']

reporte = []
reporte.append('# Informe ejecutivo - Alura Store')
reporte.append(f'_Generado: {datetime.now().strftime("%Y-%m-%d %H:%M")}_\n')
reporte.append('## Resumen de hallazgos')
reporte.append('**1) Facturación total por tienda**')
reporte.append(facturacion.to_markdown(index=False))
reporte.append('\n**2) Categorías más populares (Top) por tienda**')
reporte.append('(Ver gráficos en el cuaderno)')
reporte.append('\n**3) Calificación promedio por tienda**')
if 'ratings' in locals():
    reporte.append(ratings.to_markdown(index=False))
else:
    reporte.append('_No se encontraron reseñas (review_score) válidas en los datos._')
reporte.append('\n**4) Productos más y menos vendidos por tienda**')
reporte.append('(Ver tabla y gráficos en el cuaderno)')
reporte.append('\n**5) Costo promedio de envío por tienda**')
reporte.append(envio.to_markdown(index=False))

reporte.append('\n## Recomendación')
reporte.append('Se construyó un **índice compuesto** con pesos:')
reporte.append(f"- Facturación: {weights['revenue']*100:.0f}%")
reporte.append(f"- Unidades: {weights['units']*100:.0f}%")
reporte.append(f"- Calificación: {weights['rating']*100:.0f}%")
reporte.append(f"- Costo de envío (menor es mejor): {weights['shipping']*100:.0f}%")
reporte.append('\nLa **tienda con peor desempeño** (menor puntaje compuesto) es:')
reporte.append(f'**{peor_tienda}**.\n')
reporte.append('> Recomendación: considerar su venta para financiar el nuevo negocio, salvo que existan estrategias de mejora específicas (p.ej., optimización de envíos o reposicionamiento de categorías de baja rotación).')

report_md = "\n".join(reporte)
with open('Alura_Store_Reporte.md', 'w', encoding='utf-8') as f:
    f.write(report_md)

print('Reporte guardado en: Alura_Store_Reporte.md')
print('\nVista previa del reporte:')
print(report_md[:1200] + ('...' if len(report_md) > 1200 else ''))



<a id="10"></a>

## 10) Cumplimiento de requisitos

- **Datos**: lectura y manipulación de CSV con **Pandas**.
- **Gráficos (matplotlib)**: se incluyen **barras**, **circular (pie)** y **dispersión (scatter)** (≥ 3 tipos distintos). Además, se agrega un **histograma** opcional de reseñas.
- **Métricas analizadas**: ingresos, categorías más vendidas, reseñas, productos más/menos vendidos, costo de envío.
- **Recomendación**: texto final con la tienda sugerida para vender y justificación basada en datos.
